#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2018


# Homework 3:  Embeddings + Recommenders

### 100 points [5% of your final grade]

### Due: Monday, April 9 by 11:59pm

*Goals of this homework:* There are two main learning objectives: (i) implement and evaluate a pre-cursor to modern word2vec embeddings; and (ii) implement, evaluate, and improve upon traditional collaborative filtering recommenders.

*Submission Instructions:* To submit your homework, rename this notebook as UIN_hw#.ipynb. For example, this homework submission would be: YourUIN_hw3.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Thursday, April 12 at 11:59pm.

# Part 1: Word Embeddings (50 points)
For this first part, we're going to implement a word embedding approach that is a bit simpler than word2vec. The key idea is to look at co-occurrences between center words and context words (somewhat like in word2vec) but without any pesky learning of model parameters.

If you're interested in a deeper treatment of comparing count vs. learned embeddings, take a look at: [Don’t count, predict! A systematic comparison of
context-counting vs. context-predicting semantic vectors](
http://www.aclweb.org/anthology/P14-1023)

## Load the Brown Corpus

The dataset for this part is the (in)famous [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) that is a collection of text samples from a wide range of sources, with over one million unique words. Good for us, you can find the Brown corpus in nltk. *Make sure you have already installed nltk with something like: conda install nltk*

In [11]:
import nltk
nltk.download('brown')
nltk.download('stopwords')

[nltk_data] Downloading package brown to /Users/huangyian/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/huangyian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Once you have it locally, you can load the dataset into your notebook. You can access the words using brown.words():

In [12]:
from nltk.corpus import brown
brown.words()

[u'The', u'Fulton', u'County', u'Grand', u'Jury', ...]

## 1.1 Dataset Pre-processing
OK, now we need to do some basic pre-processing. For this part you should:

* Remove stopwords and punctuation.
* Make everything lowercase.

Then, count how often each word occurs. We will define the 5,000 most  frequent words as your vocabulary (V). We will define the 1,000 most frequent words as our context (C). Include a print statement below to show the top-20 words after pre-processing.

In [13]:
# Your Code Here...
import re
import string
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
tokenizer = RegexpTokenizer(r'\w+')
word_list = brown.words()
stop = set(stopwords.words('english')) 
preprocessed_list = [x.lower() for x in word_list]

preprocessed_list = [tokenizer.tokenize(word) for word in preprocessed_list]

dimensionConv = []
for listWord in preprocessed_list:
    for word in listWord:
        dimensionConv.append(word)
preprocessed_list = dimensionConv
preprocessed_list = [word for word in preprocessed_list if word not in stop]

# preprocessed = []
# for w in preprocessed_list:
#     if re.match('^[a-zA-Z0-9]*$', w):
#         if w not in stop:
#             preprocessed.append(w)
# preprocessed_list = preprocessed

freqMap = nltk.FreqDist(preprocessed_list)
freqList = sorted(freqMap.iteritems(), key=lambda x : (x[1], x[0]), reverse = True)
wordList = []
countFre = []
for key, val in freqList:
    countFre.append(val)
    wordList.append(key)
print "top 20 in the list:", wordList[:20]
V = wordList[:5000]
C = wordList[:1000]

top 20 in the list: [u'one', u'would', u'said', u'time', u'new', u'could', u'two', u'may', u'first', u'man', u'like', u'even', u'made', u'also', u'many', u'must', u'well', u'af', u'back', u'years']


## 1.2 Building the Co-occurrence Matrix 

For each word in the vocabulary (w), we want to calculate how often context words from C appear in its surrounding window of size 4 (two words before and two words after).

In other words, we need to define a co-occurrence matrix that has a dimension of |V|x|C| such that each cell (w,c) represents the number of times c occurs in a window around w. 

In [14]:
# Your Code Here...
import numpy
len_v, len_c = len(V), len(C)
print len_v
len_pre = len(preprocessed_list)
co_occur = [[0 for x in range(len_c)] for y in range(len_v)]
# print co_occur
for index in range(0, len_pre):
    preprocessed_words = preprocessed_list[index]
    if preprocessed_words in V:
        row = V.index(preprocessed_words)    
        left = index - 2;
        right = index + 2;
        if right >= len_pre:
            right = len_pre - 1;
        if left < 0:
            left = 0;
            
        for inner in range(left, right + 1):
            word = preprocessed_list[inner]
            if (inner != index) & (word in C):
                column = C.index(word) 
                co_occur[row][column] += 1
#         for ind_c in range(0, len_c):
#             for ind_v in range(left, right + 1):
#                 if C[ind_c] == preprocessed_list[ind_v]:
#                     co_occur[row][ind_c] = co_occur[row][ind_c] + 1
        
print "end"


5000
end


## 1.3 Probability Distribution

Using the co-occurrence matrix, we can compute the probability distribution Pr(c|w) of context word c around w as well as the overall probability distribution of each context word c with Pr(c).  

In [22]:
# Your Code Here...
import numpy as np
print len_v
# Pr(c|w)
conditional_matrix = [[0 for x in range(len_c)] for y in range(len_v)]
for i in range(0, len_v):
    sumup = 0
    for j in range(0, len_c):
        sumup = sumup + co_occur[i][j]
    for j in range(0, len_c):
        if sumup == 0:
            conditional_matrix[i][j] = 0
        else:
            conditional_matrix[i][j] = float(co_occur[i][j]) / sumup

# Pr(c)
occurence_c = []
for j in range(0, len_c):
    sumup = 0
    for i in range(0, len_v):
        sumup = sumup + co_occur[i][j]
    occurence_c.append(sumup)
    
total = 0;
pr_c = []
for j in range(0, len_c):
    total = total + occurence_c[j]
for j in range(0, len_c):
    pr_c.append(float(occurence_c[j]) / total)
    
# print conditional_matrix   
# print pr_c
print "end"


5000
end


## 1.4 Embedding Representation

Now you can represent each vocabulary word as a |C| dimensional vector using this equation:

Vector(w)= max(0, log (Pr(c|w)/Pr(c)))

This is a traditional approach called *pointwise mutual information* that pre-dates word2vec by some time. 

In [16]:
# Your Code Here...
import math
c = 0
vector_w = [[0 for x in range(len_c)] for y in range(len_v)]
for i in range(0, len_v):
    for j in range(0, len_c):
        if conditional_matrix[i][j] == 0:
            vector_w[i][j] = 0
            c = c + 1
        else:
            vector_w[i][j] = max(0, math.log(float(conditional_matrix[i][j]) / pr_c[j]))
print len_v
print len(vector_w)
# print vector_w[0]

5000
5000


## 1.5 Analysis

So now we have some embeddings for each word. But are they meaningful? For this part, you should:

- First, cluster the vocabulary into 100 clusters using k-means. Look over the words in each cluster, can you see any relation beween words? Discuss your observations.

- Second, for the top-20 most frequent words, find the nearest neighbors using cosine distance (1- cosine similarity). Do the findings make sense? Discuss.

In [17]:
# # Your Code Here...
import numpy as np
from sklearn.cluster import KMeans
from scipy import spatial
top20 = wordList[:20]
vec20 = vector_w[:20]
word_vec = np.array(vector_w)
km = KMeans(n_clusters=100, random_state=0).fit_predict(word_vec)
print "end"

end


In [18]:
for i in range(1,100):
    ind = np.where(km == i)
    cluster = []
    for j in ind[0]:
          cluster.append(V[j])
    print "Cluster", i, ":", cluster

Cluster 1 : [u'alexander']
Cluster 2 : [u'acquired']
Cluster 3 : [u'pulled']
Cluster 4 : [u'arnold', u'empire']
Cluster 5 : [u'children', u'education', u'schools', u'subjects']
Cluster 6 : [u'company', u'industrial', u'manager']
Cluster 7 : [u'pursue']
Cluster 8 : [u'figure']
Cluster 9 : [u'even', u'made', u'many', u'must', u'well', u'much', u'people', u'make', u'work', u'life', u'another', u'might', u'great', u'since', u'without', u'found', u'part', u'upon', u'every', u'course', u'always', u'fact', u'though', u'far', u'called', u'almost', u'yet', u'better', u'present', u'point', u'find', u'second', u'group', u'social', u'give', u'order', u'important', u'rather', u'case', u'among', u'often', u'god', u'best', u'need', u'church', u'become', u'power', u'family', u'least', u'seemed', u'mind', u'today', u'help', u'others', u'although', u'law', u'whole', u'problem', u'sense', u'kind', u'certain', u'thus', u'name', u'matter', u'perhaps', u'human', u'action', u'free', u'show', u'example', u'hi

Cluster 55 : [u'radiation', u'observed', u'moon', u'intensity', u'measurements', u'wave', u'observations', u'thermal', u'emission', u'temperatures', u'planets', u'planet', u'planetary', u'solar']
Cluster 56 : [u'systems']
Cluster 57 : [u'operations']
Cluster 58 : [u'analysis']
Cluster 59 : [u'james']
Cluster 60 : [u'normal']
Cluster 61 : [u'firm']
Cluster 62 : [u'generally']
Cluster 63 : [u'chief']
Cluster 64 : [u'depends']
Cluster 65 : [u'1', u'2', u'3', u'per', u'4', u'5', u'10', u'total', u'million', u'6', u'8', u'1960', u'7', u'30', u'cent', u'15', u'20', u'12', u'50', u'average', u'25', u'100', u'0', u'1959', u'9', u'11', u'degrees', u'14', u'16', u'maximum', u'500', u'24', u'april', u'40', u'60', u'compared', u'approximately', u'18', u'estimated', u'22', u'13', u'21', u'200', u'23', u'percent', u'17', u'january', u'equivalent', u'diameter', u'february', u'shares', u'pounds', u'70', u'26', u'acres', u'19', u'75', u'36', u'ratio', u'net', u'27', u'80', u'mg', u'cents', u'29', u'28'

DISCUSS: 

For this 100 clusters, we could notice the relations between the words in a single cluster. For example, the word 'two' is in the same cluster of the word 'three' and the word 'four'. The word 'state' is in the same cluster of 'united' and 'city', etc... The relationship between in words each cluster is stronger than the relationship of words from different clusters and this shows the significance of the KMean clustering. In order to find words with the strongest relationships, we need to make sure that 'k' is properly chosen. For example, if k is too large, we could get too many clusters, though the words in a single cluster is really close, we might overlook other words which should be in the same clusters so that this KMeans is useless and tell us much less information. On the contrary, if k is too small, the number of words in a cluster is large, we could not tell much information with a very small k either.

In [156]:
knn_dic = {}
for i in range(0, 20):
    knn_list = []
    res = []
    for j in range(0, len(vector_w)):
        distance = spatial.distance.cosine(vec20[i], vector_w[j])
        knn_dic[distance] = j
        knn_list.append(distance)
    knn_list.sort()
    knn = knn_list[:6]
#     print top20
    for index in range(1, len(knn)):
        res.append(wordList[knn_dic[knn[index]]])
    print "word:", top20[i]
    print "neighbors:", res

word: one
neighbors: [u'every', u'took', u'another', u'first', u'four']
word: would
neighbors: [u'could', u'might', u'want', u'think', u'anything']
word: said
neighbors: [u'know', u'tell', u'like', u'asked', u'knew']
word: time
neighbors: [u'day', u'play', u'days', u'around', u'long']
word: new
neighbors: [u'york', u'city', u'company', u'modern', u'development']
word: could
neighbors: [u'would', u'might', u'said', u'want', u'going']
word: two
neighbors: [u'three', u'four', u'several', u'six', u'five']
word: may
neighbors: [u'also', u'would', u'might', u'shall', u'possible']
word: first
neighbors: [u'second', u'next', u'last', u'every', u'home']
word: man
neighbors: [u'woman', u'young', u'girl', u'never', u'boy']
word: like
neighbors: [u'said', u'saw', u'know', u'little', u'woman']
word: even
neighbors: [u'seem', u'might', u'always', u'much', u'thought']
word: made
neighbors: [u'make', u'making', u'however', u'possible', u'act']
word: also
neighbors: [u'may', u'set', u'means', u'best', 

DISCUSS:

Unlike the Kmeans clustering, KNN method returns back even numbers inside a group of words. Inside a single group, we notice the words are related to each other. For example, the word 'man' has neighbor such as 'boy', 'woman', 'girl', etc, which totally make sense to us and the word 'two' has its neighbors 'three', 'four', 'five', etc. Kmeans is unsupervised because the words vec has no external classification and KNN combines the classification of the K nearest points.

# Part 2. Collaborative Filtering (50 points)

In this second part, you will implement collaborative filtering on the Netflix prize dataset -- don’t freak out, the provided sample dataset has only ~2000 items and ~28,000 users.

As background, read the paper [Empirical Analysis of Predictive Algorithms for Collaborative Filtering](https://arxiv.org/pdf/1301.7363.pdf) up to Section 2.1. Of course you can read further if you are interested, and you can also refer to the course slides for collaborative filtering.

## 2.1 Load Netflix Data

The dataset is subset of movie ratings data from the Netflix Prize Challenge. Download the dataset from Piazza. It contains a train set, test set, movie file, and README file. The last two files are original ones from the Netflix Prize, however; in this homework you will deal with train and test files which both are subsets of the Netflix training data. Each of train and test files has lines having this format: MovieID,UserID,Rating.

Your job is to predict a rating in the test set using those provided in the training set.

In [2]:
# load the data, then print out the number of ratings, 
# movies and users in each of train and test sets.
# Your Code Here...
import collections  
import math
import numpy as np  


content_test = []
with open("/Users/huangyian/Downloads/netflix-dataset/TestingRatings.txt") as f:
    for line in f:
        inner_list = [elt.strip() for elt in line.split(',')]
        content_test.append(inner_list)
print "number of ratings in test rating:", len(content_test)

movies_set_test = set()
users_set_test = set()
test_dictionary = collections.OrderedDict() 

for line in content_test:
    users_set_test.add(line[1])
    movies_set_test.add(line[0])
    test_dictionary[line[1]] = test_dictionary.get(line[1],{})
    test_dictionary[line[1]][line[0]] = line[2]
    
# for user_id,user_movies_rate in user_dic.iteritems():
#     for movie_id,movie_rate in user_movie_rate.iteritems():
#         print "User: ",user_id," Movie: ",movie_id
     
print "number of movies in test rating:", len(movies_set_test)
print "number of users in test rating:", len(users_set_test)

content_train = []
with open("/Users/huangyian/Downloads/netflix-dataset/TrainingRatings.txt") as f:
    for line in f:
        inner_list = [elt.strip() for elt in line.split(',')]
        content_train.append(inner_list)
print "number of ratings in training rating:", len(content_train)

users_set_train = set()
movies_set_train = set()
user_dic = collections.OrderedDict() 

for line in content_train:
    users_set_train.add(line[1])
    movies_set_train.add(line[0])

    user_dic[line[1]] = user_dic.get(line[1],{})
    user_dic[line[1]][line[0]] = line[2]
    
# for user_id,movie_id in user_dic.iteritems():
#     for user_movie_rate in user_movie_rate:
#         print "User: ",user_id," Movie: ",movie_id
    
print "number of movies in training rating:", len(movies_set_train)
print "number of users in training rating:", len(users_set_train)


number of ratings in test rating: 100478
number of movies in test rating: 1701
number of users in test rating: 27555
number of ratings in training rating: 3255352
number of movies in training rating: 1821
number of users in training rating: 28978


## 2.2 Implement CF

In this part, you will implement the basic collaborative filtering algorithm described in Section 2.1 of the paper -- that is, focus only on Equations 1 and 2 (where Equation 2 is just the Pearson correlation). You should consider the first 5,000 users with their associated items in the test set. 

Note that you should test the algorithm for a small set of users e.g., 10 users first and then run for 5,000 users. It may take long to run but you won't have memory issues. 

Set k to 0.1. 

In [3]:
def intersection(list1, list2):  
    set1 = set(list1)  
    set2 = set(list2)  
    common = set1.intersection(set2)  
    return list(common) 

numbers_users = 5000

top_user_list = list(users_set_test)[:numbers_users]
top_user_avg_dic = collections.OrderedDict() 
correlation_dic = collections.OrderedDict() 
movie_user_dic = collections.defaultdict(list)

# Get the movie to user dictionary
# Range: all lineItem in Trainning data
# for user_id,user_movies in user_dic.iteritems():
#     user_movies = user_dic.get(user_id)
#     for movieID in user_movies.iterkeys():
#         movie_user_dic[movieID].append(user_id)
# print "Finished Get movie_user_dic"
for user_id in top_user_list:
    user_movies = user_dic.get(user_id)
    for movieID in user_movies.iterkeys():
        movie_user_dic[movieID].append(user_id)
print "Finished Get movie_user_dic"
        
# Get the average vote for each user, store it in top_user_avg_dic
# Range:  all LineItem in Trainning data
for user_id,user_movies in user_dic.iteritems():
#     print "User: ",user_id," Movie: ",user_movies
    user_movie_rate_sum = sum([float(x) for x in user_movies.itervalues()])
    top_user_avg_dic[user_id] = user_movie_rate_sum/len(user_movies)
print "Finished Get User_vote_average_dic"



Finished Get movie_user_dic
Finished Get User_vote_average_dic


In [4]:
# Get the correlation between users, store it in correlation_dic
for index1 in range(0, len(top_user_list)):
    for index2 in range(index1 + 1, len(top_user_list)):
        
        user_x = list(top_user_list)[index1]
        userx_movies = user_dic.get(user_x)
        userx_movies_ID = [movie for movie in userx_movies.iterkeys()]
        
        user_y = list(top_user_list)[index2]
        usery_movies = user_dic.get(user_y)
        usery_movies_ID = [movie for movie in usery_movies.iterkeys()]
        
        common_movies = intersection(userx_movies_ID, usery_movies_ID)

        divisor = sum([(float(userx_movies.get(movie_id))-top_user_avg_dic[user_x])
                       *(float(usery_movies.get(movie_id))-top_user_avg_dic[user_y]) 
                       for movie_id in common_movies])
    
        division = np.sqrt(sum([np.square(float(userx_movies.get(movie_id))-top_user_avg_dic[user_x]) for movie_id in common_movies]) *
                  sum([np.square(float(usery_movies.get(movie_id))-top_user_avg_dic[user_y]) for movie_id in common_movies]))
        
        correlation = divisor/division if divisor != 0 else 0
#         print "Divisor is: ",divisor, " division is:", division, " correlation is: ",correlation
        
        correlation_dic.setdefault(user_x, {})[user_y] = correlation
        correlation_dic.setdefault(user_y, {})[user_x] = correlation
    
print "end"



end


## 2.3 Evaluation 

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [5]:
k = 0.001
predict_dict = collections.defaultdict(list)
count = 0
deltasum = 0
deltaSquareSum = 0

for index in range(0, 5000):
    user_id = top_user_list[index]
    movies_rates = test_dictionary.get(user_id)
    for movie_id, movie_rate in movies_rates.iteritems():
#         print "User ",user_id,"'s rate to movie ",movie_id,"is",movie_rate," His average score is",top_user_avg_dic.get(user_id)
        predict_score = top_user_avg_dic.get(user_id)    
        if not movie_user_dic.get(movie_id):
            predict_score = top_user_avg_dic.get(user_id)
        else:
            predict_score = top_user_avg_dic.get(user_id) + k * sum([(float(user_dic.get(activity_user).get(movie_id)) - top_user_avg_dic.get(activity_user)) * float(correlation_dic.get(user_id).get(activity_user)) 
                                                                     for activity_user in movie_user_dic.get(movie_id)])
        deltasum += abs(predict_score-float(movie_rate))
        deltaSquareSum += np.square(predict_score-float(movie_rate))
        count += 1
    
#         predict_dict.setdefault(user_id, {})[movie_id] = predict_score

print "Mean Absolute Error:", deltasum/count
print "Root Mean Squared Error:", np.sqrt(deltaSquareSum/count)


Mean Absolute Error: 0.744302083101
Root Mean Squared Error: 0.9393061290821592


## 2.4 Extensions

Given your results in the previous part, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach.

In [8]:
# change the value of k
k1 = 0.01
k2 = 0.1
predict_dict_k1 = collections.defaultdict(list)
count_k1 = 0
deltasum_k1 = 0
deltaSquareSum_k1 = 0
predict_dict_k2 = collections.defaultdict(list)
count_k2 = 0
deltasum_k2 = 0
deltaSquareSum_k2 = 0

for index in range(0, 5000):
    user_id = top_user_list[index]
    movies_rates = test_dictionary.get(user_id)
    for movie_id, movie_rate in movies_rates.iteritems():
#         print "User ",user_id,"'s rate to movie ",movie_id,"is",movie_rate," His average score is",top_user_avg_dic.get(user_id)
        predict_score_k1 = top_user_avg_dic.get(user_id) 
        predict_score_k2 = top_user_avg_dic.get(user_id) 
        if not movie_user_dic.get(movie_id):
            predict_score_k1 = top_user_avg_dic.get(user_id)
            predict_score_k2 = top_user_avg_dic.get(user_id)
        else:
            predict_score_k1 = top_user_avg_dic.get(user_id) + k1 * sum([(float(user_dic.get(activity_user).get(movie_id)) - top_user_avg_dic.get(activity_user)) * float(correlation_dic.get(user_id).get(activity_user)) 
                                                                     for activity_user in movie_user_dic.get(movie_id)])
            predict_score_k2 = top_user_avg_dic.get(user_id) + k2 * sum([(float(user_dic.get(activity_user).get(movie_id)) - top_user_avg_dic.get(activity_user)) * float(correlation_dic.get(user_id).get(activity_user)) 
                                                                     for activity_user in movie_user_dic.get(movie_id)])
        deltasum_k1 += abs(predict_score_k1-float(movie_rate))
        deltaSquareSum_k1 += np.square(predict_score_k1-float(movie_rate))
        count_k1 += 1
        deltasum_k2 += abs(predict_score_k2-float(movie_rate))
        deltaSquareSum_k2 += np.square(predict_score_k2-float(movie_rate))
        count_k2 += 1
    
#         predict_dict.setdefault(user_id, {})[movie_id] = predict_score
print "Mean Absolute Error of k = 0.001:", deltasum/count
print "Root Mean Squared Error of k = 0.001:", np.sqrt(deltaSquareSum/count)

print "Mean Absolute Error of k = 0.01:", deltasum_k1/count_k1
print "Root Mean Squared Error of k = 0.01:", np.sqrt(deltaSquareSum_k1/count_k1)

print "Mean Absolute Error of k = 0.1:", deltasum_k2/count_k2
print "Root Mean Squared Error of k = 0.1:", np.sqrt(deltaSquareSum_k2/count_k2)


Mean Absolute Error of k = 0.001: 0.744302083101
Root Mean Squared Error of k = 0.001: 0.9393061290821592
Mean Absolute Error of k = 0.01: 1.13569500504
Root Mean Squared Error of k = 0.01: 1.607518858361435
Mean Absolute Error of k = 0.1: 9.92803887345
Root Mean Squared Error of k = 0.1: 16.438431669880522


In [63]:
# Your Code Here...
# from sklearn.metrics.pairwise import cosine_similarity
top_user = list(users_set_test)[:num]
correlation_cosine = collections.OrderedDict() 
for index1 in range(0, len(top_user)):
    for index2 in range(index1 + 1, len(top_user)):
        
        user_x = list(top_user)[index1]
        userx_movies = user_dic.get(user_x)
        userx_movies_ID = [movie for movie in userx_movies.iterkeys()]
        
        user_y = list(top_user_list)[index2]
        usery_movies = user_dic.get(user_y)
        usery_movies_ID = [movie for movie in usery_movies.iterkeys()]
        
        common_movies = intersection(userx_movies_ID, usery_movies_ID)

        numerator = sum([(float(userx_movies.get(movie_id)))*(float(usery_movies.get(movie_id))) for movie_id in common_movies])

        
    
        denominator1 = np.sqrt(sum([np.square(float(userx_movies.get(movie_id))) for movie_id in userx_movies_ID]))
        denominator2 = np.sqrt(sum([np.square(float(usery_movies.get(movie_id))) for movie_id in usery_movies_ID]))
        denominator = denominator1 * denominator2 
        correlation = numerator/(denominator) if denominator != 0 else 0
#         print "Divisor is: ",divisor, " division is:", division, " correlation is: ",correlation
        
        correlation_cosine.setdefault(user_x, {})[user_y] = correlation
        correlation_cosine.setdefault(user_y, {})[user_x] = correlation
    
print "end"
# print correlation_cosine

end


In [67]:
predict = collections.defaultdict(list)
c = 0
d_sum = 0
squareSum = 0
movie_user = collections.defaultdict(list)

for user_id in top_user:
    user_movies = user_dic.get(user_id)
    for movieID in user_movies.iterkeys():
        movie_user[movieID].append(user_id)
for index in range(0, num):
    user_id = top_user[index]
    movies_rates = test_dictionary.get(user_id)
    for movie_id, movie_rate in movies_rates.iteritems():
#         print "User ",user_id,"'s rate to movie ",movie_id,"is",movie_rate," His average score is",top_user_avg_dic.get(user_id)
        predict_score = top_user_avg_dic.get(user_id)    
        if not movie_user.get(movie_id):
            predict = top_user_avg_dic.get(user_id)
        else:
#             print float(correlation_cosine.get(user_id).get(activity_user))
            predict = top_user_avg_dic.get(user_id) + k * sum([float(user_dic.get(activity_user).get(movie_id)) - top_user_avg_dic.get(activity_user) * float(correlation_cosine.get(user_id).get(activity_user)) for activity_user in movie_user.get(movie_id)])
        d_sum += abs(predict_score-float(movie_rate))
        squareSum += np.square(predict_score-float(movie_rate))
        c += 1
    
#         predict_dict.setdefault(user_id, {})[movie_id] = predict_score
print "cosine similarity:"
print "Mean Absolute Error:", d_sum/c
print "Root Mean Squared Error:", np.sqrt(squareSum/c)


cosine similarity:
Mean Absolute Error: 0.730877571498
Root Mean Squared Error: 0.9110322978049715


DISCUSS:
    
The prediction of a vote between an active user and a single movie is related to 'k', the correlation matrix, and the average of the user's votes. 

In order to improve RMSE and MAE, I decide to change those factors, in this case, the value of 'k' and the correlation matrix. When using k = 0.001, the MAE and RMSE reach the minimum which indicates the best results, however, when applying k = 0.01 and k = 0.1, RMSE and MAE accumulates which indicates the larger prediction error, which told us k should be chosen in considerate of the rate range of the scoring system. When applying k = 0.1 and 0.01, the range will not be within 0~5, some of the values exceed the range. 

To deal with this problem, we could either change the value of k or distort the prediction votes value a bit to meet the scoring requirement.

Also, we could use different method to calculate the correlation matrix. In this case, I changed the Pearson method to cosine similarity. Comparing with the Pearson, the mae decreased from 0.744 to 0.731, and RMSE decreases from 0.939 to 0.911.